# 导入包和函数

In [ ]:
import os
import sys
from multiprocessing import Pool, cpu_count

import numpy as np
import pandas as pd
import torch
from matplotlib import pyplot as plt


torch.manual_seed(756)
np.random.seed(756)


current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

from mfdnn import *
from utils import *

# 基础设置

In [ ]:
torch.manual_seed(756)
np.random.seed(756)

data_path = "PM25"

covariate_data = np.load(os.path.join(data_path, "interpolated_results_2022.npy")) # 365 * T1 * T2 * 6
covariate_data = np.transpose(covariate_data, (3, 0, 1, 2))  #  6 * 365 * T1 * T2
y = np.load(os.path.join(data_path, "pm25_daily_means_2022.npy"))


train_indices = pd.read_csv(os.path.join(data_path, "train_indices_list.csv"), header=None).to_numpy()
test_indices  = pd.read_csv(os.path.join(data_path, "test_indices_list.csv"), header=None).to_numpy()


n = 300             
p = 6              
frun = 50          
epsilon = 0.01     

domain_range = [
    [np.array([40.5, -74.3]), np.array([41.5, -73.7])] for _ in range(p)
]

model_params = {
    'num_basis': [5, 5],        
    'layer_sizes': [64, 64],    
    'epochs': 200,              
    'val_ratio': 0.1,           
    'patience': 10             
}


lam1_values = [0, 0.001, 0.003, 0.005, 0.007, 0.01]
lam2_values = [0, 0.001, 0.01]

# 辅助函数

In [ ]:
def select_best_hyperparameters(X_train, y_train, p, domain_range, lam1_values, lam2_values, model_params, epsilon=0.01):

    
    mse_results = np.zeros((len(lam1_values), len(lam2_values)))
    selection_info = {}

    y_train_mean = np.mean(y_train)
    y_train_std = np.std(y_train)
    
    for i, lam1 in enumerate(lam1_values):
        for j, lam2 in enumerate(lam2_values):
            try:

                train_losses, val_losses, model, l21 = MFDNN(
                    p=p, resp=y_train, func_cov=X_train,
                    num_basis=model_params['num_basis'],
                    layer_sizes=model_params['layer_sizes'],
                    domain_range=domain_range,
                    epochs=model_params['epochs'],
                    val_ratio=model_params['val_ratio'],
                    patience=model_params['patience'],
                    lam1=lam1, lam2=lam2, std_resp=True
                )
                
     
                mse_results[i, j] = min(val_losses) if len(val_losses) > 0 else np.mean(train_losses[-10:])
    
                selected_vars = [k for k, norm in enumerate(l21) if norm > epsilon]
                

                selection_info[f"{i}_{j}"] = {
                    'model': model,
                    'lam1': lam1,
                    'lam2': lam2,
                    'mse': mse_results[i, j],
                    'selected_vars': selected_vars,
                    'y_mean': y_train_mean,
                    'y_std': y_train_std,
                    'l21': l21
                }
            except Exception as e:
                mse_results[i, j] = np.inf
                selection_info[f"{i}_{j}"] = {
                    'model': None,
                    'lam1': lam1,
                    'lam2': lam2,
                    'mse': np.inf,
                    'selected_vars': [],
                    'y_mean': y_train_mean,
                    'y_std': y_train_std,
                    'l21': None
                }
    

    best_idx = np.unravel_index(np.argmin(mse_results), mse_results.shape)
    best_candidate = selection_info[f"{best_idx[0]}_{best_idx[1]}"]
    
    return best_candidate['lam1'], best_candidate['lam2'], best_candidate

def evaluate_on_test_set(best_candidate, X_test, y_test, p, domain_range, model_params):

    try:
        y_mean = best_candidate['y_mean']
        y_std = best_candidate['y_std']

        test_predictions_normalized = MFDNN_predict(p, best_candidate['model'], X_test, model_params['num_basis'], domain_range)
        test_predictions_original = test_predictions_normalized.detach().numpy() * y_std + y_mean

        test_mse = np.mean((test_predictions_original.flatten() - y_test) ** 2)
        test_rmse = np.sqrt(test_mse)
        
        return test_rmse, test_predictions_original
    except Exception as e:
        return np.inf, None

# 50次循环

## 50次循环运行

In [ ]:

all_results = {
    'test_rmse': [],
    'selection_counts': [0]*p,
    'rank_sequences': []  
}


for run_idx in range(frun):
    if run_idx % 10 == 0:
        print(f"Run {run_idx+1}/{frun}")
    
    X_train = covariate_data[:, train_indices[run_idx], :, :]
    X_test  = covariate_data[:, test_indices[run_idx], :, :]
    
    y_train = y[train_indices[run_idx]]
    y_test  = y[test_indices[run_idx]]

    lam1, lam2, best_candidate = select_best_hyperparameters(
        X_train, y_train, p=p, domain_range=domain_range,
        lam1_values=lam1_values, lam2_values=lam2_values,
        model_params=model_params, epsilon=epsilon
    )

    test_rmse, _ = evaluate_on_test_set(
        best_candidate, X_test, y_test, p=p,
        domain_range=domain_range, model_params=model_params
    )
    all_results['test_rmse'].append(test_rmse)
    

    l21 = best_candidate.get('l21', None)
    if l21 is not None:
        ranked_vars = [int(k) for k in np.argsort(-l21.detach().cpu().numpy())]
        print(f"Run {run_idx+1}, ranked_vars: {ranked_vars}")
    else:
        print(f"Run {run_idx+1}, no L21 available, skipping ranking")
        ranked_vars = None
    
    all_results['rank_sequences'].append(ranked_vars)

    for var_idx in best_candidate['selected_vars']:
        all_results['selection_counts'][var_idx] += 1



Run 1/50
Run 1, ranked_vars: [1, 0, 4, 2, 3, 5]
Run 2, ranked_vars: [1, 3, 5, 0, 4, 2]
Run 3, ranked_vars: [1, 4, 3, 5, 0, 2]
Run 4, ranked_vars: [1, 5, 0, 2, 3, 4]
Run 5, ranked_vars: [1, 0, 2, 4, 5, 3]
Run 6, ranked_vars: [0, 4, 1, 5, 3, 2]
Run 7, ranked_vars: [0, 4, 5, 3, 1, 2]
Run 8, ranked_vars: [1, 0, 5, 4, 3, 2]
Run 9, ranked_vars: [1, 5, 0, 4, 3, 2]
Run 10, ranked_vars: [1, 5, 4, 0, 2, 3]
Run 11/50
Run 11, ranked_vars: [1, 5, 4, 0, 3, 2]
Run 12, ranked_vars: [5, 0, 1, 4, 3, 2]
Run 13, ranked_vars: [1, 5, 0, 3, 4, 2]
Run 14, ranked_vars: [5, 4, 0, 2, 3, 1]
Run 15, ranked_vars: [1, 0, 5, 4, 3, 2]
Run 16, ranked_vars: [1, 5, 4, 3, 0, 2]
Run 17, ranked_vars: [0, 1, 4, 5, 2, 3]
Run 18, ranked_vars: [4, 5, 1, 3, 0, 2]
Run 19, ranked_vars: [1, 5, 4, 0, 2, 3]
Run 20, ranked_vars: [1, 0, 4, 5, 3, 2]
Run 21/50
Run 21, ranked_vars: [1, 4, 5, 0, 3, 2]
Run 22, ranked_vars: [5, 4, 1, 3, 2, 0]
Run 23, ranked_vars: [1, 5, 0, 4, 3, 2]
Run 24, ranked_vars: [1, 5, 0, 4, 2, 3]
Run 25, ranked_vars:

## 50次循环结果输出

In [ ]:

summary_results = {
    'RMSE_mean': np.mean(all_results['test_rmse']),
    'RMSE_std': np.std(all_results['test_rmse']),
    'selection_counts': all_results['selection_counts']
}

print(f"Test set RMSE Mean: {summary_results['RMSE_mean']:.4f}")
print(f"Test set RMSE Std: {summary_results['RMSE_std']:.4f}")

front3_count = np.zeros(p, dtype=int)  

for seq in all_results['rank_sequences']:
    for var in seq[:3]:  
        front3_count[var] += 1


for i, count in enumerate(front3_count):
    print(f"Predictor{i+1}: {count} times")


df_vars = pd.DataFrame({
    'Predictor': [f'X{i+1}' for i in range(p)],
    'front3_count': front3_count
})

df_rmse = pd.DataFrame({
    'Predictor': ['RMSE_mean', 'RMSE_std'],
    'front3_count': [summary_results['RMSE_mean'], summary_results['RMSE_std']]
})

df_summary = pd.concat([df_vars, df_rmse], ignore_index=True)
df_summary.to_csv("PM25_MFDNN_variable_selection_summary.csv", index=False, encoding="utf-8-sig")

测试集 RMSE 平均值: 2.5002
测试集 RMSE 标准差: 0.2220
变量在前三位置出现次数（变量 1~6）：
变量1: 29 次
变量2: 45 次
变量3: 2 次
变量4: 4 次
变量5: 31 次
变量6: 39 次
结果已保存为 PM25_MFDNN_variable_selection_summary.csv
